In [6]:
import tensorflow as tf
from einops import rearrange

tf.random.set_seed(123)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

from einops import rearrange, repeat, reduce

from darts.datasets import AirPassengersDataset

In [44]:
W_res = tf.random.normal((10, 10))
W_res = tf.where(tf.random.normal(W_res.shape) > 0.5, tf.zeros_like(W_res), W_res)
tf.math.maximum(W_res)

TypeError: maximum() missing 1 required positional argument: 'y'

In [73]:

class ESN(tf.keras.Model):
    def __init__(self, reservoir_size=100, input_size=1, output_size=1,  spectral_radius=1.0, connectivity_rate=1.0, learning_rate = 0.1, epochs=1, washout=1, activation="tanh"):
        
        super(ESN, self).__init__()
        self.reservoir_size = reservoir_size
        self.input_size = input_size
        self.epochs = epochs
        self.connectivity_rate = connectivity_rate
        self.spectral_radius = spectral_radius
        self.washout = washout
        self.output_size = output_size
        self.lr = learning_rate
        self.activation = self.activation_fn(activation)

        self.state = tf.zeros((self.reservoir_size, 1))
        self.W_in = tf.random.uniform((self.reservoir_size, self.input_size), minval=-1, maxval=1)

        self.W_out = None

        ## Initializing Reservoir Weights according to "Re-visiting the echo state property"(2012)
        ##
        ## Initialize a random matrix and induce sparsity.
        self.W_res = tf.random.normal((reservoir_size, reservoir_size))
        self.W_res = tf.where(tf.random.normal(self.W_res.shape) > self.connectivity_rate, tf.zeros_like(self.W_res), self.W_res)


        ## Scale the matrix based on user defined spectral radius.
        current_spectral_radius = tf.reduce_max(tf.abs(tf.linalg.eigvals(self.W_res)))
        self.W_res = self.W_res * (self.spectral_radius / current_spectral_radius)        

        self.all_states = [self.state]

    
    @staticmethod
    def activation_fn(x):
         
        activation_keys = ["sigmoid", "relu", "tanh"]

        if x in activation_keys:
              if x == "tanh":
                   return tf.keras.activations.tanh
              elif x == "relu":
                   return tf.keras.activations.relu()
              elif x == "sigmoid":
                   return tf.keras.activations.sigmoid()
            
        else:
            raise ValueError(f"Activation {x} does not exists")
        

    def fit(self, X_train, y_train):
        

        state_collection_matrix = tf.zeros((self.input_size + self.reservoir_size, 1))
        # self.state = np.zeros((self.reservoir_size, 1))

        ## Calculate state of reservoirs per time step
        for i in range(X_train.shape[0]-1):

            

            input = rearrange(X_train[i], 'c -> c 1')
            input_product = self.W_in@input
            state_product = self.W_res@self.state
            self.state = self.activation(input_product + state_product)
            state_collection_matrix= tf.hstack((state_collection_matrix, tf.concat((self.state, input))))

            self.all_states.append(self.state)

        ## Update W_out
        mat1 = state_collection_matrix.T[self.washout:,:]
        self.W_out = tf.linalg.lstsq(matrix=mat1, rhs=y_train[self.washout:,:])

        
        
    def predict(self, X_test):
            prediction = tf.zeros((self.output_size,1))
            for i in range(X_test.shape[0]- 1):
                input = X_test[i].reshape(-1,1)
                print(input.shape)
                input_product = self.W_in@input
                state_product = self.W_res@self.state
                self.state = self.activation(input_product + state_product)
                concat_matrix= np.concatenate((self.state, input))
                pred =  self.W_out@concat_matrix
                prediction = np.hstack([prediction, pred])

                self.all_states.append(self.state)
            
            prediction = rearrange(prediction, 'c r -> r c')
            if self.output_size == self.input_size:
                return prediction[1:,:]
            else:
                return prediction

In [74]:
time_series = AirPassengersDataset().load()
X = time_series.values()
X_train, X_test = train_test_split(X, test_size=0.2, shuffle=False)

sc = MinMaxScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [75]:
X_train_std = tf.convert_to_tensor(X_train_std)
X_test_std = tf.convert_to_tensor(X_test_std)

In [76]:
esn = ESN(reservoir_size=20, input_size=1, output_size=1, spectral_radius=0.7, connectivity_rate=0.8, washout=1)

In [77]:
esn.fit(X_train_std, X_train_std)

InvalidArgumentError: cannot compute MatMul as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:MatMul] name: 